# Pandas

The Numpy library is excellent for numerical computations, but it lacks support to handle missing data or non-omogeneous arrays. The **Pandas** library is based on Numpy and extends the Numpy functionality, and is currently one of the most widely used tools for data manipulation, providing high-performance, easy-to-use data structures and advanced data analysis tools.

In particular Pandas features:

* A fast and efficient `Series` and `DataFrame` objects for data manipulation with integrated indexing;
* Tools for reading and writing data between in-memory data structures and different formats (CSV, Excel, SQL, HDF5);
* Smart data alignment and integrated handling of missing data;
* Time series-functionalities;
* Convenient label-based slicing, fancy indexing, and subsetting of large data sets;
* Hierarchical axis indexing provides an intuitive way of working with high-dimensional data in a lower-dimensional data structure;
* Aggregating and transforming data with a powerful "group-by" engine; 
* High performance merging and joining of data sets;
* Highly optimized for performance, with critical code paths written in Cython or C.

In [ ]:
import pandas as pd # standard naming convention
import numpy as np

## Series

Pandas Series represent an extension of the Numpy 1D arrays. The content of a Series is equivalent to a Numpy array, and in addition the axis  is labeled. Labels don't need to be unique, but must be of a *hashable* type.

Since the content is of type `ndarray`, the content has to be *omogeneous*. However, there is the possibility to store heterogeneous data, but the content in this case would be of type `object`.

One of the most important examples are the **time series**, which are used to keep track of the time evolution of a certain quantity.

Link to the official Pandas Series [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html).

In [ ]:
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']

# Calling the Series constructor
# Constructor requires the data, and optionally the indices and data type
sr = pd.Series(np.arange(10)*0.5, index=tuple(letters[:10]), dtype=float)
print("series:\n", sr, '\n')
print("series type:\n", type(sr), '\n')
print("indices:\n", sr.index, '\n')
print("values:", sr.values, type(sr.values), '\n') # values of the Series are actually a numpy array
print("type:\n", sr.dtype, '\n')

In [ ]:
print("element by index:", sr['f'], '\n') # Accessing elements like arrays
print("element by attribute:", sr.f, '\n') # Accessing elements like attributes - not recommended

# selecting a subset of the series
subsr = sr[['d', 'f', 'h']] # note the double square brackets
print("series subset:\n", subsr, type(subsr), '\n') # Multiple indexing returns another series

In [ ]:
# Extracting elements and operations are the same as numpy array (slicing)
print(sr[:3], '\n')
print(sr[7:], '\n')
print(sr[::3], '\n')

# Fancy indexing works on Series, too
print(sr[sr > 3], '\n')

# You can also pass Series to numpy funtions
print(np.exp(sr), '\n')
print("Series mean:", np.mean(sr), ", std:", np.std(sr), '\n')

Series may contain non-omogeneous data; in this case, the data type is referred to as `object`. Non-homogeneous data is normally handeled also by Pandas and does not represent a problem, however this pays the price of less time-efficient operations.

In [ ]:
# Series can be created from a python dictionary, too
# Note that the elements can be of different types
d = {'b' : 1, 'a' : 'cat', 'c' : [2, 3]}
so = pd.Series(d) # alternative constructor that taks a dict as the only input
print(so, '\n')

A key difference between Pandas Series and Numpy arrays is that operations between Series **automatically align the data based on the label**.

Thus, you can write operations without considering whether the Series involved have the same labels, or even the same size.

If there is no matching element, the resulting value would be a `NaN`.

In [ ]:
s = pd.Series(np.arange(5), index=tuple(letters[:5]))
print("series:\n", s, '\n')

s1 = s[1:]
print("shifted series:\n", s1, '\n')

s2 = s1 + s
print("shifted sum:\n", s2, '\n')

s3 = s1 + s[:-1]
print("double shifted sum:\n", s3, '\n')

### Time series

**Datetime**

When dealing with time, Python provides the `datetime` library that allows to store the date and time in an dedicated object, which possess several methods to access the relevant quantities (day, month, year, hours, minutes, seconds, ...)

In [ ]:
# To define a date, the datetime module is very useful
import datetime as dt

date = dt.date.today()
print("Today's date:", date)

# specify year, month, day, hour, minutes, seconds, and microseconds
date = dt.datetime(2020, 11, 12, 10, 45, 10, 15)
print("Date and time:", date)
print("Month:", date.month, "and minutes:", date.minute)

**Pandas Timestamps**

Timestamped data is the most basic type of time series data that associates values with points in time.

Functions like `pd.to_datetime` can be used to convert between different formats and, for instance, when reading the time stored as a string from a dataset:

In [ ]:
# Get the timestamp, which is the nanoseconds from January 1st 1970 (Unix time)
tstamp = pd.Timestamp(date)
#tstamp = pd.Timestamp(dt.datetime(1970, 1, 1, 0, 0, 0, 1))
print("Timestamp:", tstamp.value)

# when creating a timestamp the format can be explicitly passed
ts = pd.to_datetime('2010/11/12', format='%Y/%m/%d')
print("Time:", ts, ", timestamp:", ts.value, ", type:", type(ts))

ts = pd.to_datetime('12-11-2010 10:39', format='%d-%m-%Y %H:%M')
print("Time:", ts, ", timestamp:", ts.value, ", type:", type(ts))

**Pandas Date range**

Time series are very often used to describe the behaviour of a quantity as a function of time. Pandas has a special type of index for that, `DatetimeIndex`, that can be created e.g. with the function `pd.data_range()`.

In [ ]:
# create DatetimeIndex using ranges:
days = pd.date_range(date, periods=7, freq='D')
print("7 days range:", days)

seconds = pd.date_range(date, periods=3600, freq='s')
print("1 hour in seconds:", seconds)

To learn more about the frequency strings, please check the [documentation](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases).

A standard series can be created, and (a range of) elements can be used as indices:

In [ ]:
print("index:\n", days, '\n')
tseries = pd.Series(np.random.normal(10, 1, len(days)), index=days)
print("time series:\n", days, '\n')
# Extracting elements
print("slice by position:\n", tseries[0:4], '\n')
print("slice by date range:\n", tseries['2020-9-11' : '2020-9-14'], '\n') # note that includes end time

`pd.to_datetime` can also be used to create a `DatetimeIndex` if the argument is a list:

In [ ]:
print(pd.to_datetime([1, 2, 3, 4], unit='D', origin=pd.Timestamp('1980-02-03')))

## DataFrame

A pandas DataFrame can be thought as a tabular spreadsheet, although the performance, the functionalities and the capabilities are very different.

Similarly to Series, the DataFrame structure also contains labeled axes (rows and columns). Arithmetic operations **align on both row and column labels**. Each column in a DataFrame is a Series object: as a matter of fact, a DataFrame can be thought of as a dict-like container for Series objects.

The elements can be of all types, and missing data could be present too (represented as `NaN`).

For future reference (or for people already familiar with R), a pandas DataFrame is also similar to the R DataFrame.

Link to the official [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

### Constructor

A DataFrame objects can be created by passing a dictionary of objects. Note that the dictionary values are not omogeneous and do not have the same length. In these cases, pandas will automatically adjust the sizes, by replicating the content or adding NaN if necessary.

In [ ]:
df = pd.DataFrame({
    'A' : 1.,
    'B' : pd.Timestamp('20130102'),
    'C' : pd.Series(3, index=range(4), dtype='float32'),
    'D' : np.arange(7, 11),
    'E' : pd.Categorical(["test", "train", "test", "train"]), # a Series that represents a category label
})
# the keys of the dictionary represent the labels of the columns
# since no index is specified, the simplest one [0, 1, 2, ...] is added by Pandas automatically

df

An example of DataFrame with a `DatatimeIndex` object as index:

In [ ]:
entries = 10
columns = ['A', 'B', 'C', 'D']
dates = pd.date_range('11/9/2020 14:45:00', freq='h', periods=entries) # days/month/year
df = pd.DataFrame(np.random.randn(entries, len(columns)), index=dates, columns=columns)
df # pay attention that the date is printed as year-day-month

### Viewing Data

In [ ]:
df.head()

In [ ]:
df.tail(4)

In [ ]:
df.index

In [ ]:
df.columns

In [ ]:
df.values

In [ ]:
df.describe()

In [ ]:
df.T

In [ ]:
df.sort_index(axis=1, ascending=False)

In [ ]:
df.sort_values(by="C", ascending=False)

### Selection

#### Slicing

DataFrame slicing allows to select a subset of the DataFrame, or an entire column (a Series):

In [ ]:
# standard and safe
print(df['A'], '\n', type(df['A']), '\n') # Returns a Series (a column)

# equivalent but dangerous (imagine blank spaces in the name of the column, or a column named "T")
print(df.A, '\n')

Numpy-like slicing by row ranges is possible, and usually returns a **view** of the original DataFrame:

In [ ]:
# selecting rows by range. Returns another DataFrame (usually a view)
print(df[0:3], '\n')

# or by index range
print(df["2020-11-09 14:45:00" : "2020-11-09 16:45:00"])

#### Selection by label

The most common way to select elements, rows, or columns in a DataFrame is by using the `.loc[]` method.

`.loc` supports multi-indexing, and usually returns a **copy** of the DataFrame.

In [ ]:
# getting a part of the DataFrame (in this case, a row)) using a label. Returns a Series
dfs = df.loc[dates[0]] # equivalent to df.loc[df.index[0]]
print(dfs, '\n', type(dfs), '\n')

In [ ]:
# selecting on a multi-axis by label:
dfa = df.loc[:, ['A','B']]
dfa

In [ ]:
# showing label slicing, both endpoints are included:
df.loc['2020-11-09 18:45:00':'2020-11-09 20:45:00', ['A','B']]

In [ ]:
# getting an individual element
print(df.loc[dates[1], 'A'], '\n', type(df.loc[dates[1], 'A']), '\n')

The `.at()` method is equivalent to `.loc[]`. Use `at` if you only need to get or set a single value in a DataFrame or Series.

In [ ]:
print(df.at[dates[1], 'A'])

#### Selecting by position

`.iloc[]` is similar to `.loc[]`, but instead of labels, it uses pure integer-location based indexing for selection by position.

But differently from `.loc[]`, `.iloc[]` usually returns a **view**, not a copy.

In [ ]:
# select via the position of the passed integers:
print(df.iloc[3], '\n', type(df.iloc[3]), '\n')

If you specify just one axis or idex, a Series is returned. If you specify both axis or indices, you get a DataFrame instead:

In [ ]:
# row and column ranges selected with numpy-like notation:
dfv = df.iloc[3:5, 0:2]
print(dfv, '\n', type(df.iloc[3:5, 0:2]), '\n')

In [ ]:
# selection of multiple elements with lists
df.iloc[[1, 2, 4], [0, 2]] # selecting rows 1,2 and 4 for columns 0 and 2

In [ ]:
# slicing rows explicitly
df.iloc[1:3, :]

# slicing columns explicitly
df.iloc[:, 1:3]

Similary to `.loc[]` and `.at[]`, there is also `.iat[]` alongside `.iloc[]`:

In [ ]:
# selecting an individual element by position: no difference between iloc and iat
print(df.iloc[1,1], ", type:", type(df.iloc[1,1]))
print(df.iat[1,1], ", type:", type(df.iat[1,1]))

#### Masks

Boolean masks can be used in the same way as Numpy, and they represent a very powerful way of filtering out data with certain features. Just like Numpy fancy indexing, using a mask usually returns a **copy** of the DataFrame.

In [ ]:
# Selecting on the basis of boolean conditions applied to the whole DataFrame
dfc = df[df > 0]
dfc.iat[0, 0] = -99
# a DataFrame with the same shape is returned, with NaN's where condition is not met
# Note that when a NaN is present in a column of integers, the column (Series) is casted to float
dfc

In [ ]:
# Filter by a boolean condition on the values of a single column
mask = dfc['B'] < 0.5
mask

In [ ]:
# Filter only the rows that correspond to a True in the Series used as mask
dfc[mask]

**Queries**

Pandas uses a database-like engine to select elements according to a query on the columns of the DataFrame:

In [ ]:
dfq = df.query('C > 0.5')
dfq

which is equivalent to `dfq = df[df['C'] > 0.5]`:

In [ ]:
dfw = df[df['C'] > 0.5]
dfw

### Copy and views in DataFrames

The view/copy behaviour in Pandas is not as easy as it may appear, as there are counter-intuitive exceptions. There was a plan to fix this by quite some time, but a fix has not been deployed yet.

Check this discussion [here](https://www.practicaldatascience.org/html/views_and_copies_in_pandas.html):

    In numpy, the rules for when you get views and when you don’t are a little complicated, but they are consistent: certain behaviors (like simple indexing) will always return a view, and others (fancy indexing) will never return a view.

    But in pandas, whether you get a view or not—and whether changes made to a view will propagate back to the original DataFrame—depends on the structure and data types in the original DataFrame.


In summary, there is only one way to write safe code when dealing with slides of a dataframe: after every instruction that selects a subset of a DataFrame, force the copy by appending `.copy()` to the slice.

### Assignement

Assignment is typically performed after a selection:

In [ ]:
# Make sure to copy the DataFrame if you plan to modify it, and you don't want to change the original object
dfa = df.copy()

# setting values by label (same as by position)
dfa.at[dates[0], 'A'] = -1

# setting and assigning a numpy array
dfa['D'] = np.array([5] * len(dfa))

# defining a brand new column by means of a pd.Series: indexes must be the same!
dfa['E'] = pd.Series(np.arange(len(dfa))*2, index=dfa.index)

# using masks for assigment
dfa[dfa < 0] = -dfa

dfa

### Dropping

Dropping columns is an example of **a method that does not modify the original object**, and returns a new modified object. In other words, if you want to keep the modified DataFrame, perform a new assignment:

```python
df = df.drop(...)
```
Alternatively, the modification of the original object can be forced by specifying `inplace=True` among the arguments.

In [ ]:
dfb = dfa.copy()

# Dropping by column..
dfb.drop(['E'], axis=1)
dfb

As you can see, there is no effect on the original object. That's because  new object is returned instead. To keep it, there are two alternatives:

In [ ]:
dfc = dfb.drop(columns=['E'])
dfc

In [ ]:
dfb.drop(columns=['E'], inplace=True) # equivalent to the previous one, but the original object has been replace inplace
dfb

### Dealing with missing data

Pandas primarily uses the value `np.nan` to represent missing data. It is by default not included in computations. If there is a `NaN` entry in a Series of integers, the type of the Series will be changed to floats.

In [ ]:
df_wNan = dfb[dfb > 0.5]
df_wNan

In [ ]:
# dropping rows with at least a Nan
df_wNan.dropna(how='any')
df_wNan

In [ ]:
# getting a mask
df_wNan.isna()
# df_wNan.notna()

In [ ]:
# filling missing data (not recommended, unless you really mean it)
df_wNan.fillna(value=0)

### Operations

Operations on the elements of a DataFrame are quite straightforward, as the syntax is the same as the one used for Series. Also for DataFrames, operations are performed between elements that share the same labels. Operations on columns are extremly fast, almost as fast as the actual operation between elements in a row.

In [ ]:
# Some statistics (mean() just as an example)
# on rows
print(df.mean(axis=0), '\n')
# on columns
print(df.mean(axis=1), '\n')

In [ ]:
# Global operations on columns
df.apply(np.sum) # or whatever function defined by the user

In [ ]:
# Also lambda functions are accepted
df.apply(lambda x: x - x.max())

In [ ]:
# syntax is as usual similar to that of numpy arrays
df['S'] = df['A'] + df['C']
df

### Application of a function: apply vs transform

User-defined or standard functions can be applied on entire DataFrames or columns, with very short execution times.

There are two main methods, `apply()` and `transform()`:

In [ ]:
def dcos(theta):
    theta = theta * (np.pi / 180)
    return np.cos(theta)

# Apply method with custom function
dfa['cosine'] = dfa["E"].apply(dcos)

# Transform method with lambda function
dfa['EplusOne'] = dfa["E"].transform(lambda x: x + 1)
dfa

The major differences between `apply` and `transform` are:

   - Input: `apply` passes all the columns to the custom function, while `transform` passes each column.
   - Output: the custom function passed to `apply` can return a scalar, or a Series or DataFrame, while the custom function passed to `transform` must return a sequence (a Series, array or list) with the same length.

In summary, `transform` works on just one Series, and `apply` works on the entire DataFrame.

### Merge

Pandas provides various functions for easily combining together Series and DataFrames in join / merge-type operations.

**Concat**

Concatenation (adding rows) is straightforward:

In [ ]:
rdf = pd.DataFrame(np.arange(40).reshape(10, 4))
rdf

In [ ]:
# split DataFrame into 3 pieces, row-wise
pieces = [rdf[:3], rdf[3:7], rdf[7:]]
print(pieces, '\n')
pieces[2]

In [ ]:
# put it back together
pd.concat(pieces)

# in this case, indices are already set; if they are not, indices can be ignored
#pd.concat(pieces, ignore_index=True)

In case of dimension mismatch, `NaN` are added where needed.

**Merge/Join**

SQL-like operations on table can be performed on DataFrames. This is a quite advanced use case, refer to the [doc](https://pandas.pydata.org/pandas-docs/stable/merging.html#merging) for more info/examples.

In [ ]:
left = pd.DataFrame({'key': ['foo', 'bar'], 'lval': [1, 2]})
right = pd.DataFrame({'key': ['foo', 'bar'], 'rval': [4, 5]})

pd.merge(left, right, on="key")

### Grouping

In real world applications, it's quite common that several entries (row) belong to a certain entity, or "group". DataFrames have a powerful tool to perform operations on entries of the same group. The method is called `.groupby()`, and it usually involves one or more of the following steps:

* Splitting the data into groups based on some criteria
* Applying a function to each group independently
* Combining the results into a data structure


In [ ]:
gdf = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar', 'foo', 'foo'],
                    'B' : [1, 1, 2, 3, 2, 2, 1, 3],
                    'C' : np.arange(8),
                    'D' : np.linspace(10, -10, 8)})
gdf

In [ ]:
# Grouping and then applying the sum() 
# function to the resulting groups (effective only where numerical values are present)
gdf.groupby('A').sum()

In [ ]:
# Example: find maximum value in column D for each group, and assign the value to a new column
gdf['M'] = gdf.groupby('A')['D'].transform(np.max)
gdf

### Multi-indexing

Hierarchical / Multi-level indexing allows sophisticated data analysis on higher dimensional data. In practice, it enables you to store and manipulate data with an arbitrary number of dimensions in lower dimensional data structures like Series (1D) and DataFrames (2D).

In [ ]:
# Creat multi-dimensional index
tuples = list(zip(['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']))
multi_index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
print(multi_index, '\n', type(multi_index), '\n')

# Create multi-indexed dataframe or series
s = pd.Series(np.arange(8)/np.pi, index=multi_index)
s

In [ ]:
# multi-indexing enables further features of the groupby method,
# e.g. when group-by by multiple columns
gdf.groupby(['A', 'B']).sum()

## Summary: a demonstration of the efficiency of the DataFrame

Let's go the hard way and load a (relatively) large dataset with approximately 1 million rows:

In [ ]:
# Uncomment to download the file. Run the command just once
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

In [ ]:
file_name = "./data/data_000637.txt"
data = pd.read_csv(file_name)
data

Let's now do some operations among (elements of) columns

In [ ]:
itime = dt.datetime.now()
print("Begin time:", itime)

# the one-liner command
data['WEIGHTEDSUM'] = data['TDC_CHANNEL'] * 2.1 + data['BX_COUNTER'] * 0.1 + 2

ftime = dt.datetime.now()
print("End time:", ftime)
print("Elapsed time:", ftime - itime)

data

In [ ]:
# the loop
def conversion(data):
    result = []
    for i in range(len(data)): 
        result.append(data.loc[data.index[i], 'TDC_CHANNEL'] * 2.1 + data.loc[data.index[i], 'BX_COUNTER'] * 0.1 + 2)
    return result

itime = dt.datetime.now()
print("Begin time:", itime)
data['WEIGHTEDSUM'] = conversion(data)
ftime = dt.datetime.now()
print("End time:", ftime)
print("Elapsed time:", ftime - itime)

data